In [ ]:
import whisper
from tqdm import tqdm

# -----------------------------
# 2.1 Cargar el modelo Whisper
# -----------------------------
# Puedes usar "small", "base", "medium", "large", etc.
# Los modelos más grandes ofrecen mayor precisión pero tardan más.
modelo = whisper.load_model("small")  # o "base", "medium", "large"

# -----------------------------
# 2.2 Función para transcribir un archivo de video directamente
# -----------------------------
def transcribir_con_whisper(video_path, output_txt_path):
    """
    Transcribe un archivo MP4 ó WAV usando Whisper y guarda el texto en output_txt_path.
    El "model.transcribe" se encarga de extraer audio internamente si el input es video.
    """

    # 1) Whisper retorna un diccionario con keys: 'text', 'segments', 'language', etc.
    print(f"▶️ Iniciando transcripción de {video_path} con Whisper...")
    resultado = modelo.transcribe(video_path, verbose=False)

    # 2) Extraemos el texto completo (un string largo)
    texto_completo = resultado["text"].strip()

    # 3) Guardamos en un .txt
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write(texto_completo)

    print(f"✅ Transcripción finalizada y guardada en: {output_txt_path}")
    return texto_completo

# -----------------------------
# 2.3 Uso de la función
# -----------------------------
if __name__ == "__main__":
    # Supongamos que tus archivos MP4 están en tu directorio actual de Colab
    video1 = "trump1.mp4"
    video2 = "trump2.mp4"
    video3 = "trump3.mp4"

    # Transcribe y guarda cada uno
    transcribir_con_whisper(video1, "trump1.txt")
    transcribir_con_whisper(video2, "trump2.txt")
    transcribir_con_whisper(video3, "trump3.txt")


100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 54.2MiB/s]


▶️ Iniciando transcripción de trump1.mp4 con Whisper...
Detected language: English


100%|██████████| 179199/179199 [01:09<00:00, 2569.23frames/s]


✅ Transcripción finalizada y guardada en: trump1.txt
▶️ Iniciando transcripción de trump2.mp4 con Whisper...
Detected language: English


100%|██████████| 349869/349869 [03:59<00:00, 1460.98frames/s]


✅ Transcripción finalizada y guardada en: trump2.txt
▶️ Iniciando transcripción de trump3.mp4 con Whisper...
Detected language: English


100%|██████████| 292870/292870 [03:02<00:00, 1605.98frames/s]

✅ Transcripción finalizada y guardada en: trump3.txt


In [19]:
from transformers import DistilBertForSequenceClassification
m = DistilBertForSequenceClassification.from_pretrained("modelos_cloud/distilbert_multicount")
print("num_labels del modelo:", m.config.num_labels)
# Debe imprimir: 3


num_labels del modelo: 3


In [17]:
!rm -rf modelos_cloud/distilbert_multicount

In [18]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
import math
import pickle
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

# -----------------------------
# 1. Definición de palabras objetivo
# -----------------------------
# Puedes ajustar esta lista según las palabras que te interese predecir.
PALABRAS_OBJETIVO = ["china", "economy", "border"]

# -----------------------------
# 2. Utilidad para cargar y tokenizar un archivo .txt
# -----------------------------
def cargar_palabras(ruta_txt):
    """
    Lee un archivo de texto, lo convierte a minúsculas, elimina saltos de línea
    y signos de puntuación básicos, y retorna una lista de palabras tokenizadas.
    """
    with open(ruta_txt, "r", encoding="utf-8") as f:
        texto = f.read().lower()

    # Quitar signos de puntuación comunes para tokenizar más limpio
    for ch in ['.', ',', ';', ':', '!', '?', '"', '“', '”', '(', ')', '[', ']', '—', '…']:
        texto = texto.replace(ch, "")
    palabras = texto.split()
    return palabras

# -----------------------------
# 3. Dataset para entrenamiento con DistilBERT (regresión)
# -----------------------------
class TrumpDataset(Dataset):
    """
    Dataset para entrenamiento de DistilBERT en regresión:
      - Cada elemento corresponde a una posición i en el discurso.
      - Input: texto completo hasta la palabra i (tokenizado y truncado).
      - Label: vector de conteo futuro [cnt_china, cnt_economy, cnt_border].
    """
    def __init__(self, palabras, tokenizer, max_length=512):
        """
        palabras: lista de tokens (por ejemplo, ['the', 'economy', 'is', ...])
        tokenizer: DistilBERT tokenizer
        max_length: longitud máxima que DistilBERT tomará como contexto
        """
        self.palabras = palabras
        self.N = len(palabras)
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Precalcular las posiciones de cada palabra objetivo en todo el discurso
        self.posiciones = {
            w: [idx for idx, pw in enumerate(palabras) if pw == w]
            for w in PALABRAS_OBJETIVO
        }
        # Construimos la lista de índices 0..N-1
        self.indices = list(range(self.N))

    def __len__(self):
        return self.N

    def _conteo_futuro(self, i, w):
        """
        Retorna cuántas veces aparece w en posiciones > i,
        usando la lista precalculada self.posiciones[w].
        """
        # Las posiciones en self.posiciones[w] están ordenadas por construcción,
        # pero para simplicidad aquí filtramos directamente.
        return sum(1 for pos in self.posiciones[w] if pos > i)

    def __getitem__(self, idx):
        """
        Para un índice idx (posición i), retorna:
          - input_ids, attention_mask: tensores para DistilBERT
          - labels: tensor con conteo futuro [cnt_china, cnt_economy, cnt_border]
        """
        i = self.indices[idx]

        # Reconstruimos el texto completo hasta la posición i (inclusive)
        texto_hasta_i = " ".join(self.palabras[: i + 1])

        # Tokenizamos con DistilBERT (truncando/padding a max_length)
        enc = self.tokenizer(
            texto_hasta_i,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Construimos el vector de conteo futuro para cada palabra objetivo
        etiquetas = np.array(
            [self._conteo_futuro(i, w) for w in PALABRAS_OBJETIVO],
            dtype=np.float32
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(etiquetas)
        }

# -----------------------------
# 4. Clasificador DistilBERT para regresión multi−output
# -----------------------------
class DistilBERTForMultiCount(torch.nn.Module):
    """
    Envuelve DistilBERTForSequenceClassification para regresión multi−conteo:
      - num_labels = cantidad de palabras objetivo (en este caso, 3).
      - problem_type = "regression" para que use MSELoss internamente.
    """
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.bert = DistilBertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=3,
            problem_type="regression"
        )

    def forward(self, input_ids, attention_mask, labels=None):
        # DistilBERTForSequenceClassification ya calcula loss si labels no es None
        return self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

# -----------------------------
# 5. Crear DataLoader para entrenamiento
# -----------------------------
def crear_dataloaders(tokenizer, batch_size=8, max_length=512):
    """
    Lee 'trump1.txt' y 'trump3.txt', concatena sus palabras y crea un DataLoader.
    """
    palabras1 = cargar_palabras("trump1.txt")
    palabras3 = cargar_palabras("trump3.txt")
    palabras_train = palabras1 + palabras3

    dataset_train = TrumpDataset(palabras_train, tokenizer, max_length=max_length)

    loader_train = DataLoader(
        dataset_train,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )
    return loader_train

def entrenar_modelo_cloud(
    model_name="distilbert-base-uncased",
    output_dir="modelos_cloud",
    epochs=3,
    batch_size=8,
    max_length=512,
    lr=5e-5
):
    """
    Entrena DistilBERT para regresión de conteo sobre trump1+trump3,
    usando num_labels = len(PALABRAS_OBJETIVO). Guarda el modelo en output_dir.
    """
    # 1) Cargar tokenizer y DataLoader
    print("📥 Cargando tokenizer y DataLoader…")
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
    train_loader = crear_dataloaders(tokenizer, batch_size=batch_size, max_length=max_length)

    # 2) Instanciar el modelo con num_labels = número de palabras objetivo (3)
    print("🤖 Instanciando DistilBERTForSequenceClassification con regresión (num_labels=3)…")
    model = DistilBertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(PALABRAS_OBJETIVO),  # → aquí va 3
        problem_type="regression"
    )

    # 3) Definir argumentos de entrenamiento:
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=lr,
        logging_steps=100,
        save_steps=500,
        save_total_limit=2,
        remove_unused_columns=False,
        fp16 = True if torch.cuda.is_available() else False,
        no_cuda = False if torch.cuda.is_available() else True,
        report_to=[]  # deshabilita wandb
    )

    # 4) Data collator (sin cambios)
    def collate_fn(batch):
        input_ids = torch.stack([item["input_ids"] for item in batch])
        attention_mask = torch.stack([item["attention_mask"] for item in batch])
        labels = torch.stack([item["labels"] for item in batch])
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

    # 5) Crear Trainer de Hugging Face
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_loader.dataset,
        data_collator=collate_fn
    )

    # 6) Entrenar
    print("🚀 Entrenamiento iniciado…")
    trainer.train()

    # 7) Guardar el modelo y tokenizer
    os.makedirs(output_dir, exist_ok=True)
    model_path = os.path.join(output_dir, "distilbert_multicount")
    trainer.save_model(model_path)          # guarda config, pytorch_model.bin, etc.
    tokenizer.save_pretrained(model_path)   # guarda el tokenizer
    print(f"✅ Modelo cloud guardado en: {model_path}")

    return model_path

# -----------------------------
# 7. Función de simulación en “tiempo real” (cloud)
# -----------------------------
def simular_tiempo_real_cloud(
    model_path,
    texto_prueba_path="trump2.txt",
    max_length=512,
    device="cuda" if torch.cuda.is_available() else "cpu"
):
    """
    Carga el modelo distilBERT entrenado y simula palabra a palabra sobre 'trump2.txt'.
    Para cada índice i, calcula:
      - λ (conteo esperado) para cada palabra objetivo
      - P(≥1 aparición) = 1 - exp(-λ)
      - count_so_far (conteo acumulado) para cada palabra objetivo
    Imprime un diccionario con esos valores en cada paso.
    """
    # 7.1 Cargar tokenizer y modelo
    print("📥 Cargando modelo y tokenizer desde:", model_path)
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
    # Para cargar el modelo, usamos la misma clase que definimos
    model = DistilBERTForMultiCount(model_path, num_labels=len(PALABRAS_OBJETIVO))
    model.to(device)
    model.eval()

    # 7.2 Leer y tokenizar trump2.txt (lista de palabras)
    palabras2 = cargar_palabras(texto_prueba_path)
    N = len(palabras2)
    print(f"🔢 Discurso de prueba tiene {N} palabras.")

    # 7.3 Precalcular posiciones de cada palabra objetivo en trump2
    posiciones = {
        w: [idx for idx, pw in enumerate(palabras2) if pw == w]
        for w in PALABRAS_OBJETIVO
    }

    # 7.4 Inicializar contadores acumulados
    count_so_far = { w: 0 for w in PALABRAS_OBJETIVO }

    # 7.5 Recorrer palabra por palabra
    for i, w_actual in enumerate(tqdm(palabras2, desc="Simulando en Cloud")):
        # 7.5.1 Reconstruir texto hasta la posición i
        texto_hasta_i = " ".join(palabras2[: i + 1])

        # 7.5.2 Tokenizar (truncamos/pad a max_length)
        enc = tokenizer(
            texto_hasta_i,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        # 7.5.3 Forward pass (sin gradiente)
        with torch.no_grad():
            outputs = model(
                input_ids=enc["input_ids"],
                attention_mask=enc["attention_mask"]
            )
            # outputs.logits: tensor de forma [1, num_labels]
            lambdas = outputs.logits.squeeze(0).cpu().numpy()  # array de shape (3,)

        # 7.5.4 Para cada palabra objetivo, calcular probabilidad
        resultados = {}
        for idx_w, w in enumerate(PALABRAS_OBJETIVO):
            lambda_w = max(0.0, float(lambdas[idx_w]))  # asegurar λ ≥ 0
            prob_w = 1 - math.exp(-lambda_w)             # P(≥1 aparición)
            resultados[w] = {
                "lambda_esperado": lambda_w,
                "prob_al_menos_una": prob_w,
                "count_so_far": count_so_far[w]
            }

        # 7.5.5 Construir diccionario de salida y mostrarlo
        salida = {
            "indice_actual": i,
            "palabra_actual": w_actual,
            "predicciones": resultados
        }
        print(salida)

        # 7.5.6 Actualizar count_so_far si la palabra actual es objetivo
        if w_actual in count_so_far:
            count_so_far[w_actual] += 1

    print("✅ Simulación cloud en tiempo real completada.")

# -----------------------------
# 8. Bloque principal: entrenar + simular
# -----------------------------
if __name__ == "__main__":
    # 8.1 Entrenar el modelo (descomenta si aún no lo has entrenado)
    # Ajusta epochs, batch_size y max_length según tu GPU/tiempo disponible.
    modelo_guardado = entrenar_modelo_cloud(
        model_name="distilbert-base-uncased",
        output_dir="modelos_cloud",
        epochs=2,
        batch_size=8,
        max_length=256,
        lr=3e-5
    )

    # 8.2 Simular “tiempo real” sobre trump2.txt (usa el modelo entrenado)
    simular_tiempo_real_cloud(
        model_path=os.path.join(modelo_guardado),
        texto_prueba_path="trump2.txt",
        max_length=256
    )


📥 Cargando tokenizer y DataLoader…
🤖 Instanciando DistilBERTForSequenceClassification con regresión (num_labels=3)…


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 Entrenamiento iniciado…


Step,Training Loss
100,9.436900
200,6.357400
300,6.242700
400,6.152900
500,6.450300
600,6.045900
700,6.284000
800,6.212100
900,5.959100
1000,6.043800


✅ Modelo cloud guardado en: modelos_cloud/distilbert_multicount
📥 Cargando modelo y tokenizer desde: modelos_cloud/distilbert_multicount
🔢 Discurso de prueba tiene 8605 palabras.


...

{'indice_actual': 8593, 'palabra_actual': 'much', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_esperado': 5.0377912521362305, 'prob_al_menos_una': 0.9935119370010014, 'count_so_far': 0}, 'border': {'lambda_esperado': 1.468713402748108, 'prob_al_menos_una': 0.7697785029326921, 'count_so_far': 3}}}
{'indice_actual': 8594, 'palabra_actual': 'pennsylvania', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_esperado': 5.0377912521362305, 'prob_al_menos_una': 0.9935119370010014, 'count_so_far': 0}, 'border': {'lambda_esperado': 1.468713402748108, 'prob_al_menos_una': 0.7697785029326921, 'count_so_far': 3}}}
{'indice_actual': 8595, 'palabra_actual': 'and', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_es

Simulando en Cloud: 100%|█████████▉| 8601/8605 [05:00<00:00, 16.38it/s]

{'indice_actual': 8597, 'palabra_actual': 'bless', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_esperado': 5.0377912521362305, 'prob_al_menos_una': 0.9935119370010014, 'count_so_far': 0}, 'border': {'lambda_esperado': 1.468713402748108, 'prob_al_menos_una': 0.7697785029326921, 'count_so_far': 3}}}
{'indice_actual': 8598, 'palabra_actual': 'america', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_esperado': 5.0377912521362305, 'prob_al_menos_una': 0.9935119370010014, 'count_so_far': 0}, 'border': {'lambda_esperado': 1.468713402748108, 'prob_al_menos_una': 0.7697785029326921, 'count_so_far': 3}}}
{'indice_actual': 8599, 'palabra_actual': 'thank', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_espe

Simulando en Cloud: 100%|██████████| 8605/8605 [05:00<00:00, 28.63it/s]

{'indice_actual': 8601, 'palabra_actual': 'very', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_esperado': 5.0377912521362305, 'prob_al_menos_una': 0.9935119370010014, 'count_so_far': 0}, 'border': {'lambda_esperado': 1.468713402748108, 'prob_al_menos_una': 0.7697785029326921, 'count_so_far': 3}}}
{'indice_actual': 8602, 'palabra_actual': 'much', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_esperado': 5.0377912521362305, 'prob_al_menos_una': 0.9935119370010014, 'count_so_far': 0}, 'border': {'lambda_esperado': 1.468713402748108, 'prob_al_menos_una': 0.7697785029326921, 'count_so_far': 3}}}
{'indice_actual': 8603, 'palabra_actual': 'thank', 'predicciones': {'china': {'lambda_esperado': 6.730527877807617, 'prob_al_menos_una': 0.998806097442896, 'count_so_far': 5}, 'economy': {'lambda_esperado

In [20]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
stats_cloud.py

Este script recorre la transcripción trump2.txt palabra por palabra usando el modelo
DistilBERT fine-tuneado para regresión de conteo (ubicado en modelos_cloud/distilbert_multicount),
midiendo latencia de inferencia, prediciendo λ (conteo esperado) para cada palabra objetivo,
calculando el conteo real de apariciones futuras y, al final, reportando estadísticas
(MAE, RMSE, latencia promedio) para cada palabra.

Importante:
  - El modelo debe haberse entrenado con el mismo número de etiquetas (num_labels) que
    la longitud de PALABRAS_OBJETIVO.
  - Si tu modelo fue entrenado con otro num_labels, reentrena con `num_labels = len(PALABRAS_OBJETIVO)`.

Requisitos:
  - Python 3.7+
  - pip install torch transformers numpy scikit-learn tqdm
  - trump2.txt (transcripción ya tokenizada en minúsculas).
  - Un modelo DistilBERT entrenado en `modelos_cloud/distilbert_multicount/`, con num_labels = len(PALABRAS_OBJETIVO).
"""

import os
import time
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

# -----------------------------
# 1. Configuración general
# -----------------------------
# Carpeta donde se guardó el modelo Cloud (DistilBERT regresión)
MODEL_CLOUD_DIR = "modelos_cloud/distilbert_multicount"
# Ruta a la transcripción de prueba (texto tokenizado en minúsculas)
TRANSCRIPCION_PATH = "trump2.txt"
# Lista de palabras objetivo (su longitud debe coincidir con num_labels del modelo)
PALABRAS_OBJETIVO = ["china", "economy", "border"]


# -----------------------------
# 2. Función para cargar y tokenizar un archivo .txt
# -----------------------------
def cargar_palabras(ruta_txt):
    """
    Lee un archivo .txt, lo convierte a minúsculas, elimina signos de puntuación básicos,
    y retorna una lista de palabras tokenizadas.
    """
    with open(ruta_txt, "r", encoding="utf-8") as f:
        texto = f.read().lower()
    for ch in ['.', ',', ';', ':', '!', '?', '"', '“', '”', '(', ')', '[', ']', '—', '…']:
        texto = texto.replace(ch, "")
    palabras = texto.split()
    return palabras


# -----------------------------
# 3. Función para precalcular posiciones de cada palabra objetivo
# -----------------------------
def precalcular_posiciones(palabras, palabras_objetivo):
    """
    Dada la lista completa de palabras y la lista de palabras objetivo,
    retorna un diccionario {w: [idx1, idx2, ...]} con las posiciones
    donde aparece w en 'palabras'.
    """
    posiciones = {w: [] for w in palabras_objetivo}
    for idx, w in enumerate(palabras):
        if w in posiciones:
            posiciones[w].append(idx)
    return posiciones


# -----------------------------
# 4. Función para calcular conteos futuros reales
# -----------------------------
def conteo_futuro_en_pos(posiciones, i):
    """
    Dado el diccionario de posiciones y un índice i, retorna un dict {w: count}
    con el conteo de apariciones de cada palabra objetivo en posiciones > i.
    """
    remaining = {}
    for w, lista_idx in posiciones.items():
        cnt = sum(1 for p in lista_idx if p > i)
        remaining[w] = cnt
    return remaining


# -----------------------------
# 5. Función principal
# -----------------------------
def main():
    # 5.1. Cargar la transcripción de prueba
    print("1) Cargando transcripción desde:", TRANSCRIPCION_PATH)
    palabras2 = cargar_palabras(TRANSCRIPCION_PATH)
    N = len(palabras2)
    print(f"   → Discurso de prueba tiene {N} palabras tokenizadas.\n")

    # 5.2. Precalcular posiciones de cada palabra objetivo
    print("2) Precalculando posiciones de cada palabra objetivo en trump2.txt …")
    posiciones2 = precalcular_posiciones(palabras2, PALABRAS_OBJETIVO)

    # 5.3. Cargar tokenizer y modelo DistilBERT fine-tuneado
    print("3) Cargando DistilBERT y tokenizer desde:", MODEL_CLOUD_DIR)
    tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_CLOUD_DIR)
    model = DistilBertForSequenceClassification.from_pretrained(MODEL_CLOUD_DIR)

    # Verificamos cuántas etiquetas espera el modelo
    num_labels_model = model.config.num_labels
    num_objetivos = len(PALABRAS_OBJETIVO)
    if num_labels_model != num_objetivos:
        raise ValueError(
            f"El modelo fue entrenado con num_labels={num_labels_model}, "
            f"pero PALABRAS_OBJETIVO tiene longitud {num_objetivos}.\n"
            "Asegúrate de que ambos coincidan. Por ejemplo, reentrena el modelo con:\n"
            f"    DistilBertForSequenceClassification.from_pretrained(..., num_labels={num_objetivos}, problem_type='regression')"
        )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()
    print(f"   → Modelo cargado en dispositivo: {device}\n")

    # 5.4. Vectores para recopilar datos
    y_true = {w: [] for w in PALABRAS_OBJETIVO}   # conteos reales futuros
    y_pred = {w: [] for w in PALABRAS_OBJETIVO}   # lambdas predichos
    latencies_ms = []                            # latencia de inferencia por token en ms

    # 5.5. Diccionario para conteos acumulados hasta i-1
    count_so_far = {w: 0 for w in PALABRAS_OBJETIVO}

    print("4) Iniciando simulación en tiempo real (Cloud) …")
    # Iterar palabra a palabra
    for i, w_actual in enumerate(tqdm(palabras2, desc="Simulando en Cloud")):
        # 5.5.1. Calcular conteos futuros reales (antes de inferencia)
        remaining_counts = conteo_futuro_en_pos(posiciones2, i)
        for w in PALABRAS_OBJETIVO:
            y_true[w].append(remaining_counts[w])

        # 5.5.2. Reconstruir texto completo hasta la posición i (inclusive)
        texto_hasta_i = " ".join(palabras2[: i + 1])

        # 5.5.3. Tokenizar con DistilBERT (truncar/pad a max_length)
        # Asegúrate de usar el mismo max_length que en el entrenamiento (p. ej. 256).
        enc = tokenizer(
            texto_hasta_i,
            truncation=True,
            padding="max_length",
            max_length=256,
            return_tensors="pt"
        ).to(device)

        # 5.5.4. Inferencia y medición de latencia
        t0 = time.time()
        with torch.no_grad():
            outputs = model(
                input_ids=enc["input_ids"],
                attention_mask=enc["attention_mask"]
            )
            # outputs.logits: tensor de shape [1, num_labels_model]
            logits = outputs.logits.squeeze(0).cpu().numpy()  # array shape (num_labels_model,)
        t1 = time.time()

        latency_ms = (t1 - t0) * 1000.0
        latencies_ms.append(latency_ms)

        # 5.5.5. Procesar cada palabra objetivo (acorde a num_labels_model)
        for idx_w, w in enumerate(PALABRAS_OBJETIVO):
            lambda_w = float(logits[idx_w])
            # Asegurarse que λ ≥ 0
            lambda_w = lambda_w if lambda_w > 0 else 0.0
            y_pred[w].append(lambda_w)

        # 5.5.6. Actualizar count_so_far si la palabra actual es objetivo
        if w_actual in count_so_far:
            count_so_far[w_actual] += 1

    print("   → Simulación completada.\n")

    # 5.6. Calcular métricas (MAE, RMSE, latencia promedio) por palabra objetivo
    records = []
    for w in PALABRAS_OBJETIVO:
        true_arr = np.array(y_true[w])
        pred_arr = np.array(y_pred[w])

        mae = mean_absolute_error(true_arr, pred_arr)
        rmse = np.sqrt(mean_squared_error(true_arr, pred_arr))
        avg_latency = np.mean(latencies_ms)

        records.append({
            "Word": w,
            "MAE_Cloud": mae,
            "RMSE_Cloud": rmse,
            "AvgLatencyPerToken_Cloud_ms": avg_latency
        })

    df_cloud = pd.DataFrame(records)
    print("===== Resultados Estadísticos (Cloud) =====")
    print(df_cloud.to_string(index=False))

    # 5.7. Guardar resultados a CSV (opcional)
    salida_csv = "stats_cloud_results.csv"
    df_cloud.to_csv(salida_csv, index=False)
    print(f"\n   Resultados guardados en: {salida_csv}")


if __name__ == "__main__":
    main()


1) Cargando transcripción desde: trump2.txt
   → Discurso de prueba tiene 8605 palabras tokenizadas.

2) Precalculando posiciones de cada palabra objetivo en trump2.txt …
3) Cargando DistilBERT y tokenizer desde: modelos_cloud/distilbert_multicount
   → Modelo cargado en dispositivo: cuda

4) Iniciando simulación en tiempo real (Cloud) …


Simulando en Cloud: 100%|██████████| 8605/8605 [04:06<00:00, 34.87it/s]


   → Simulación completada.

===== Resultados Estadísticos (Cloud) =====
   Word  MAE_Cloud  RMSE_Cloud  AvgLatencyPerToken_Cloud_ms
  china   4.128953    4.490082                    10.084376
economy   5.049099    5.049631                    10.084376
 border   1.366774    1.414595                    10.084376

   Resultados guardados en: stats_cloud_results.csv


In [3]:
# 1.1 Instalar dependencies (solo la primera vez)
#!pip install git+https://github.com/openai/whisper.git ffmpeg-python --quiet

# 1.2 Importar librerías necesarias
import subprocess
import time
import whisper

def get_video_duration(path: str) -> float:
    """
    Devuelve la duración del video en segundos usando ffprobe.
    """
    try:
        result = subprocess.check_output([
            "ffprobe", "-v", "error",
            "-show_entries", "format=duration",
            "-of", "default=noprint_wrappers=1:nokey=1",
            path
        ], stderr=subprocess.STDOUT).decode().strip()
        return float(result)
    except Exception as e:
        raise RuntimeError(f"No se pudo obtener duración de '{path}': {e}")

def measure_whisper_latency(video_path: str, model_name: str = "small"):
    """
    Transcribe con Whisper (Cloud) y mide:
      - t_total: tiempo total de transcripción (s)
      - dur_video: duración del video (s)
      - lat_s_por_s: latencia (s transcripción / s video)
    """
    # 1) Obtener duración del video
    duración = get_video_duration(video_path)

    # 2) Cargar modelo Whisper
    print(f"🔄 Cargando modelo Whisper '{model_name}' …")
    modelo = whisper.load_model(model_name)

    # 3) Transcribir y medir tiempo
    print(f"⏱ Transcribiendo '{video_path}' con Whisper …")
    t0 = time.time()
    _ = modelo.transcribe(video_path, verbose=False)
    t1 = time.time()

    t_total = t1 - t0
    lat_s_por_s = t_total / duración if duración > 0 else float("nan")

    return t_total, duración, lat_s_por_s

# 1.3 Ejecutar la medición para trump2.mp4 con modelo "small"
video_file = "trump2.mp4"   # Asegúrate de que exista en Colab
modelo_whisper = "small"    # Puedes usar "base", "medium", "large", etc.

t_total, dur_video, lat_s_por_s = measure_whisper_latency(video_file, modelo_whisper)

print("\n=== Resultados Whisper (Cloud) ===")
print(f"Latencia total       : {t_total:.2f} s")
print(f"Duración del video   : {dur_video:.2f} s")
print(f"Latencia por segundo : {lat_s_por_s:.4f} s/s")

🔄 Cargando modelo Whisper 'small' …


100%|████████████████████████████████████████| 461M/461M [00:02<00:00, 196MiB/s]


⏱ Transcribiendo 'trump2.mp4' con Whisper …


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: English


100%|██████████| 349869/349869 [43:24<00:00, 134.31frames/s]


=== Resultados Whisper (Cloud) ===
Latencia total       : 2631.42 s
Duración del video   : 3498.68 s
Latencia por segundo : 0.7521 s/s
